In [1]:
import numpy as np 
import pandas as pd 
import pickle

from FDSSC import models, utils
from datasets.datasets import LocationChecker, FDSSCDataset
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader

In [2]:
datadir = "data/stacked_images"
labeldir = "siteData/allData/"
checker = LocationChecker(datadir, size=(9, 9), rigorous=False)
trainLandsat8Data = FDSSCDataset(datadir, labeldir+"train.csv", lochecker=checker)
valLandsat8Data = FDSSCDataset(datadir, labeldir+"val.csv", lochecker=checker)
testLandsat8Data = FDSSCDataset(datadir, labeldir+"test.csv", lochecker=checker)
trainLoader = DataLoader(trainLandsat8Data, batch_size=16, shuffle=True, num_workers=1)
valLoader = DataLoader(valLandsat8Data, batch_size=16, num_workers=1)
testLoader = DataLoader(testLandsat8Data, batch_size=16, num_workers=1)
#id2idx = utils.get_id2idx(pd.read_csv(csv), "MAJOR_COMMODITY_CODE")
id2idx = pickle.load(open("data/id2idxMasked.p", "rb"))
idx2id = {value: key for key, value in id2idx.items()}

In [3]:
# setting training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# initializing model and loss and optim
model = models.FerDSSC_model((1, 9, 9, 11), len(list(id2idx.keys())))
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#putting model on training device
model.to(device)

FerDSSC_model(
  (input_spec_conv): Conv3d(1, 24, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  (spectral_conv1): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(24, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (spectral_conv2): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(36, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (spectral_conv3): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(48, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (bn_prelu1): Bn

In [4]:
epochs = 10
frequency = 5
# setting up callbacks
checkpointFolder = "TrainingCheckpoints/1stRun/"
cbs = [utils.validate_callback(model, valLoader, loss_fn, device),
        utils.saving_checkpoints_callback(checkpointFolder, model, optimizer, frequency, epochs)]

trainingLosses = []

losses, CBouts = utils.train(model, trainLoader, epochs=epochs, loss_fn=loss_fn,
                            optimizer=optimizer, callbacks=cbs)

trainingLosses.extend(losses)



======== Epoch 1 / 10 ========
Training...
  Batch   155  of    156.    Loss: 2.12     Elapsed: 0:02:44.
  Average training loss: 2.16
  Training epoch took: 0:02:45
Running Validation...
  Loss: 2.09
  Validation took: 0:00:22

======== Epoch 2 / 10 ========
Training...
  Batch   155  of    156.    Loss: 1.71     Elapsed: 0:02:44.
  Average training loss: 1.79
  Training epoch took: 0:02:44
Running Validation...
  Loss: 1.79
  Validation took: 0:00:23

======== Epoch 3 / 10 ========
Training...
  Batch   155  of    156.    Loss: 1.59     Elapsed: 0:02:50.
  Average training loss: 1.79
  Training epoch took: 0:02:50
Running Validation...
  Loss: 1.79
  Validation took: 0:00:22

======== Epoch 4 / 10 ========
Training...
  Batch   155  of    156.    Loss: 1.48     Elapsed: 0:02:48.
  Average training loss: 1.79
  Training epoch took: 0:02:48
Running Validation...
  Loss: 1.79
  Validation took: 0:00:22

======== Epoch 5 / 10 ========
Training...
  Batch   155  of    156.    Loss: 2.04 

KeyboardInterrupt: 